In [6]:
# Best 버전으로 제출 파일 생성
best_version = results_df.iloc[0]['version']
best_summaries = versions_to_test[best_version]

print("="*80)
print(f"🎯 최고 성능: {best_version}")
print("="*80)

# 상위 2개 버전 파일 생성
for idx in range(min(2, len(results_df))):
    version_name = results_df.iloc[idx]['version']
    summaries = versions_to_test[version_name]
    
    # 파일명
    if version_name == 'conservative':
        output_name = 'submit_solar_v4_microtuned_conservative'
    elif version_name == 'moderate':
        output_name = 'submit_solar_v4_microtuned_moderate'
    elif version_name == 'quality_check':
        output_name = 'submit_solar_v4_microtuned'
    else:
        output_name = f'submit_solar_v4_{version_name}'
    
    output_path = f'./prediction/{output_name}.csv'
    
    submission = v4[['fname']].copy()
    submission['summary'] = summaries
    submission.to_csv(output_path, index=False)
    
    avg_len = sum(len(s.split()) for s in summaries) / len(summaries)
    
    print(f"\n✅ [{idx+1}] {output_name}.csv")
    print(f"   버전: {version_name}")
    print(f"   평균 길이: {avg_len:.1f} 단어")
    print(f"   Combined: {results_df.iloc[idx]['Combined']:.4f}")
    print(f"   ROUGE-2: {results_df.iloc[idx]['R2']:.2f}")
    if idx == 0:
        print(f"   ⭐ 권장 - 리더보드 먼저 제출")

print("\n" + "="*80)
print("📋 제출 가이드")
print("="*80)
print(f"🎯 목표: 52.0점 돌파")
print(f"📈 전략: v3 성공 사례 (51.8026 → 51.9421, +0.14점) 적용")
print(f"\n1️⃣ [권장] {results_df.iloc[0]['version']} 버전 먼저 제출")
print(f"   예상: 51.95~52.10점")
print(f"\n2️⃣ [백업] 점수 확인 후 다른 버전 시도")
print(f"   v3_microtuned (51.9421점) 이상 목표")

🎯 최고 성능: original_v4

✅ [1] submit_solar_v4_original_v4.csv
   버전: original_v4
   평균 길이: 16.9 단어
   Combined: 34.5376
   ROUGE-2: 19.24
   ⭐ 권장 - 리더보드 먼저 제출

✅ [2] submit_solar_v4_microtuned_conservative.csv
   버전: conservative
   평균 길이: 16.8 단어
   Combined: 34.5376
   ROUGE-2: 19.24

📋 제출 가이드
🎯 목표: 52.0점 돌파
📈 전략: v3 성공 사례 (51.8026 → 51.9421, +0.14점) 적용

1️⃣ [권장] original_v4 버전 먼저 제출
   예상: 51.95~52.10점

2️⃣ [백업] 점수 확인 후 다른 버전 시도
   v3_microtuned (51.9421점) 이상 목표


## 5. 제출 파일 생성

In [5]:
rouge = load("rouge")

print("📊 Dev셋 ROUGE 평가\n")

versions_to_test = {
    'original_v4': v4['summary'].tolist(),
    'conservative': tuned_conservative,
    'moderate': tuned_moderate,
    'quality_check': tuned_quality,
}

results = {}

for name, summaries in versions_to_test.items():
    print(f"평가 중: {name}...")
    
    scores = rouge.compute(predictions=summaries, references=dev_df['summary'].tolist())
    
    results[name] = {
        'rouge1': scores['rouge1'] * 100,
        'rouge2': scores['rouge2'] * 100,
        'rougeL': scores['rougeL'] * 100,
        'combined': (scores['rouge1'] + scores['rouge2'] + scores['rougeL']) / 3 * 100
    }

# 결과 정리
print("\n" + "="*80)
print("📈 ROUGE 평가 결과")
print("="*80 + "\n")

results_df = pd.DataFrame([
    {
        'version': k,
        'R1': v['rouge1'],
        'R2': v['rouge2'],
        'RL': v['rougeL'],
        'Combined': v['combined']
    }
    for k, v in sorted(results.items(), key=lambda x: x[1]['combined'], reverse=True)
])

print(results_df.to_string(index=False))

print(f"\n🏆 Best: {results_df.iloc[0]['version']}")
print(f"   Combined: {results_df.iloc[0]['Combined']:.4f}")
print(f"   ROUGE-2: {results_df.iloc[0]['R2']:.2f}")
print(f"\n🎯 목표: v3_microtuned 수준 (51.9421점)")

📊 Dev셋 ROUGE 평가

평가 중: original_v4...
평가 중: conservative...
평가 중: moderate...
평가 중: quality_check...

📈 ROUGE 평가 결과

      version        R1        R2        RL  Combined
  original_v4 44.512429 19.244723 39.855509 34.537554
 conservative 44.512429 19.244723 39.855509 34.537554
     moderate 44.512429 19.244723 39.855509 34.537554
quality_check 44.683964 18.985034 39.920951 34.529983

🏆 Best: original_v4
   Combined: 34.5376
   ROUGE-2: 19.24

🎯 목표: v3_microtuned 수준 (51.9421점)


## 4. Dev셋 ROUGE 평가

In [4]:
# 주요 변화 케이스 확인
print("🔍 주요 변화 샘플 (처음 5개):\n")

changed_indices = []
for i in range(len(v4)):
    if v4.iloc[i]['summary'] != tuned_quality[i]:
        changed_indices.append(i)

for idx in changed_indices[:5]:
    orig = v4.iloc[idx]['summary']
    tuned = tuned_quality[idx]
    
    print(f"[{v4.iloc[idx]['fname']}]")
    print(f"원본 ({len(orig.split())} 단어):")
    print(f"  {orig}")
    print(f"조정 ({len(tuned.split())} 단어):")
    print(f"  {tuned}")
    print(f"변화: {len(orig.split())} → {len(tuned.split())} ({len(tuned.split()) - len(orig.split()):+d} 단어)\n")

🔍 주요 변화 샘플 (처음 5개):

[test_0]
원본 (38 단어):
  #Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모를 준비해 달라고 요청합니다. 이 메모는 모든 통신이 이메일과 공식 메모로만 이루어져야 하며, 근무 시간 동안 즉시 메시지 프로그램 사용은 금지된다는 내용입니다. 이 정책은 모든 통신에 적용되며, 위반 시 경고 후 시
조정 (28 단어):
  #Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모를 준비해 달라고 요청합니다. 이 메모는 모든 통신이 이메일과 공식 메모로만 이루어져야 하며, 근무 시간 동안 즉시 메시지 프로그램 사용은 금지된다는 내용입니다.
변화: 38 → 28 (-10 단어)

[test_36]
원본 (35 단어):
  Simon은 내일 핵무기 확산을 막기 위한 시위에 참여할 예정이며, Cleo에게 함께 가자고 제안합니다. Cleo는 시위에 대해 부정적인 입장을 가지고 있습니다. Simon은 시위가 평화로울 것이라고 설명하지만, Cleo는 여전히 반대합니다. Cleo는 대신 Simon에게 정치학 수업 학생들에게 물어보라고
조정 (28 단어):
  Simon은 내일 핵무기 확산을 막기 위한 시위에 참여할 예정이며, Cleo에게 함께 가자고 제안합니다. Cleo는 시위에 대해 부정적인 입장을 가지고 있습니다. Simon은 시위가 평화로울 것이라고 설명하지만, Cleo는 여전히 반대합니다.
변화: 35 → 28 (-7 단어)

[test_41]
원본 (18 단어):
  #Person1#은 박싱 데이 세일에서 어떤 선물을 포기할지 고민하며 #Person2#와 대화하고 있습니다. #Person2#는 #Person1#이 자신의 게임을 방해하고 있다고 불평합니다.
조정 (16 단어):
  #Person1#은 박싱 데이 세일에서 어떤 선물을 포기할지 고민하며 #Person2#와 대화합니다. #Person2#는 #Person1#이 자신의 게임을 방해한다고 불평합니다.
변화: 18 → 16 (-2 단어)



## 3. 샘플 비교 (품질 확인)

In [3]:
# 각 버전 생성
print("🔬 미세조정 버전 생성 중...\n")

tuned_conservative = v4['summary'].apply(micro_tune_v4_conservative).tolist()
tuned_moderate = v4['summary'].apply(micro_tune_v4_moderate).tolist()
tuned_quality = v4['summary'].apply(micro_tune_v4_quality_check).tolist()

# 통계 비교
v4_len = v4_lengths.mean()

cons_len = sum(len(s.split()) for s in tuned_conservative) / len(tuned_conservative)
mod_len = sum(len(s.split()) for s in tuned_moderate) / len(tuned_moderate)
qual_len = sum(len(s.split()) for s in tuned_quality) / len(tuned_quality)

print("📊 버전별 통계:\n")
print(f"  [원본 v4]              평균: {v4_len:5.1f} 단어")
print(f"  [conservative]         평균: {cons_len:5.1f} 단어 ({cons_len - v4_len:+.1f})")
print(f"  [moderate]             평균: {mod_len:5.1f} 단어 ({mod_len - v4_len:+.1f})")
print(f"  [quality_check]        평균: {qual_len:5.1f} 단어 ({qual_len - v4_len:+.1f})")
print(f"\n  [목표: v3_micro]       평균: {v3_lengths.mean():5.1f} 단어")

# 변화 케이스 확인
changed_cons = sum(1 for i in range(len(v4)) if v4.iloc[i]['summary'] != tuned_conservative[i])
changed_mod = sum(1 for i in range(len(v4)) if v4.iloc[i]['summary'] != tuned_moderate[i])
changed_qual = sum(1 for i in range(len(v4)) if v4.iloc[i]['summary'] != tuned_quality[i])

print(f"\n🔄 변화된 케이스:")
print(f"  conservative: {changed_cons}개")
print(f"  moderate: {changed_mod}개")
print(f"  quality_check: {changed_qual}개")

🔬 미세조정 버전 생성 중...

📊 버전별 통계:

  [원본 v4]              평균:  16.9 단어
  [conservative]         평균:  16.8 단어 (-0.1)
  [moderate]             평균:  16.8 단어 (-0.1)
  [quality_check]        평균:  16.4 단어 (-0.5)

  [목표: v3_micro]       평균:  15.6 단어

🔄 변화된 케이스:
  conservative: 28개
  moderate: 38개
  quality_check: 62개


## 2. 미세조정 적용 및 비교

In [2]:
def micro_tune_v4_conservative(summary: str) -> str:
    """
    v4 보수적 미세조정 (v3_microtuned 전략 적용)
    - 품질 유지하면서 ROUGE-2 개선
    - 문장 구조 보존
    """
    # 1) 기본 정리
    summary = re.sub(r'\s+', ' ', summary).strip()
    
    # 2) 중복 조사 제거 (같은 조사 연속 사용)
    summary = re.sub(r'에게\s+에게', '에게', summary)
    summary = re.sub(r'에서\s+에서', '에서', summary)
    summary = re.sub(r'에\s+에\s', '에 ', summary)
    
    # 3) 동사 형태 통일 (자연스러움 유지)
    # "~라고 말합니다" → "~라고 합니다" (한 문장에 여러 번 나오는 경우만)
    말합니다_count = summary.count('말합니다')
    if 말합니다_count > 1:
        summary = summary.replace('말합니다', '합니다', 말합니다_count - 1)
    
    # 4) 과도한 진행형 단순화
    summary = re.sub(r'하고\s+있습니다', '합니다', summary)
    summary = re.sub(r'하고\s+있다', '한다', summary)
    
    # 5) 불필요한 접속사 제거 (문장 흐름 해치지 않는 범위)
    # "그리고"가 문장 시작에만 있으면 제거
    summary = re.sub(r'\.(\s+)그리고\s+', r'.\1', summary)
    
    # 6) 최종 공백 정리
    summary = re.sub(r'\s+', ' ', summary).strip()
    summary = re.sub(r'\s([,.!?])', r'\1', summary)
    
    return summary


def micro_tune_v4_moderate(summary: str) -> str:
    """
    v4 중간 강도 미세조정
    - conservative 기능 + 추가 최적화
    """
    summary = micro_tune_v4_conservative(summary)
    
    # 추가: 중복 부사 제거
    summary = re.sub(r'매우\s+많이', '많이', summary)
    summary = re.sub(r'정말\s+많이', '많이', summary)
    
    # 추가: "~이라고 말합니다" 통일
    summary = re.sub(r'이라고\s+말합니다', '이라고 합니다', summary)
    summary = re.sub(r'라고\s+말합니다', '라고 합니다', summary)
    
    return summary


def micro_tune_v4_quality_check(summary: str) -> str:
    """
    최종 품질 검증
    - 불완전한 수정 롤백
    """
    summary = micro_tune_v4_moderate(summary)
    
    # 문장이 조사/어미로 제대로 끝나는지 확인
    if not summary.endswith(('.', '!', '?', '다', '요', '습니다', '니다')):
        # 마지막 완전한 문장까지만
        sentences = re.split(r'(?<=[.!?])\s+', summary)
        if len(sentences) > 1:
            summary = ' '.join(sentences[:-1])
    
    return summary.strip()


print("✅ 3단계 미세조정 함수 정의 완료")

✅ 3단계 미세조정 함수 정의 완료


## 1. v3 성공 전략 분석

v3 → v3_microtuned 에서 적용한 미세조정:
1. 중복 표현 최소화 (바이그램 중복)
2. 문장 간 연결어 최적화
3. 불필요한 수식어 제거 (의미는 보존)
4. 문장 구조 검증 (완전성 유지)

**핵심**: 품질을 유지하면서 ROUGE-2 개선

In [1]:
import pandas as pd
import re
from evaluate import load

# 파일 로드
v4 = pd.read_csv('./prediction/submit_solar_v4.csv')
v3_micro = pd.read_csv('./prediction/submit_solar_v3_microtuned.csv')
dev_df = pd.read_csv('./data/dev.csv')

print("📊 파일 로드 완료")
print(f"  v4 (신규): {len(v4)}개")
print(f"  v3_microtuned (51.9421점): {len(v3_micro)}개")
print(f"  dev: {len(dev_df)}개")

# 현재 상태 분석
v4_lengths = v4['summary'].apply(lambda x: len(str(x).split()))
v3_lengths = v3_micro['summary'].apply(lambda x: len(str(x).split()))

print(f"\n📈 길이 비교:")
print(f"  v3_microtuned (51.9421점): 평균 {v3_lengths.mean():.1f} 단어")
print(f"  v4 (신규): 평균 {v4_lengths.mean():.1f} 단어")
print(f"  차이: {v4_lengths.mean() - v3_lengths.mean():+.1f} 단어 (v4가 약간 더 김)")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


📊 파일 로드 완료
  v4 (신규): 499개
  v3_microtuned (51.9421점): 499개
  dev: 499개

📈 길이 비교:
  v3_microtuned (51.9421점): 평균 15.6 단어
  v4 (신규): 평균 16.9 단어
  차이: +1.2 단어 (v4가 약간 더 김)


# submit_solar_v4.csv 보수적 미세조정
## v3 성공 전략 적용: 51.9421 → 52.0+ 목표

**전략**: v3_microtuned 성공 사례 분석
- v3.csv → v3_microtuned.csv: **+0.14점 향상** (51.8026 → 51.9421)
- 방법: 보수적 미세조정 (중복 최소화, 문장 완성도 유지)

**목표**: v4에 동일 전략 적용 → **52.0점 돌파**